In [1]:
from transformers import TrainingArguments
from trl import SFTTrainer

In [1]:
from huggingface_hub import login, notebook_login
notebook_login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/liam/.cache/huggingface/token
Login successful


In [3]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from datasets import Dataset
import os

# get all .md files in the statements/ directory
md_files = [os.path.join('statements', fn) for fn in os.listdir('statements') if fn.endswith('.md')]

# create loaders for each .md file
loaders = [UnstructuredMarkdownLoader(fn) for fn in md_files]

all_documents = []

for loader in loaders:
    print("Loading raw document..." + loader.file_path)
    md_doc = loader.load()
    updated_md_doc = filter_complex_metadata(md_doc)
    # Extract text content from each document
    text_content = [doc.page_content for doc in updated_md_doc]
    all_documents.extend(text_content)

# Create a Hugging Face dataset
training_set = Dataset.from_dict({"text": all_documents})

print(f"Number of documents: {len(training_set)}")

Loading raw document...statements/DUK_10-Q.md
Loading raw document...statements/META_10-K.md
Loading raw document...statements/K_10-K.md
Loading raw document...statements/MMM_10-K.md
Loading raw document...statements/AXP_10-K.md
Loading raw document...statements/JNJ_10-K.md
Loading raw document...statements/GM_10-K.md
Loading raw document...statements/ADBE_10-Q.md
Loading raw document...statements/CL_10-K.md
Loading raw document...statements/NVDA_10-Q.md
Loading raw document...statements/MSFT_10-Q.md
Loading raw document...statements/F_10-Q.md
Loading raw document...statements/NEE_10-K.md
Loading raw document...statements/VZ_10-K.md
Loading raw document...statements/AMZN_10-Q.md
Loading raw document...statements/CSCO_10-Q.md
Loading raw document...statements/AAPL_10-K.md
Loading raw document...statements/SCHW_10-Q.md
Loading raw document...statements/WFC_10-K.md
Loading raw document...statements/BA_10-K.md
Loading raw document...statements/JPM_10-K.md
Loading raw document...statements/

In [4]:
system_message = """
You are an assistant that summarizes and extracts key figures from financial statements.
Use the following context to report key figures and datapoints that would be of use for other applications. Do this in a concise manner,
focusing on the most important figures and datapoints.

Then answer these three questions in a concise manner:

1) What are the key takeaways from the context?
2) What key figures should focused on and what could be potentially drawn from them?
3) What are steps for further analysis?

Make sure to include specific details in your answers. here is the context:

{context}"""

# Define the create_conversation function
def create_conversation(sample):
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": sample["text"]}
        ]
    }

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype="auto",
  device_map="cuda",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

In [7]:
args = TrainingArguments(
    evaluation_strategy="no",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    fp16=True,
    max_steps=-1,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_steps=10,
    output_dir="phi3-mini-instruct-4k-edgar-summarization",
    optim="paged_adamw_32bit",
    lr_scheduler_type="linear"
)

/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=peft_config,
    train_dataset=training_set,
    eval_dataset=None,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/liam/VSCodeProjects/EDGAR-Summarizat

In [9]:
import gc

del model
del tokenizer

gc.collect()

torch.cuda.empty_cache()

In [10]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


Step,Training Loss
10,1.620800
20,1.471700
30,1.594800
40,1.470900
50,1.524600
60,1.687000
70,1.451400
80,1.636300
90,1.578400
100,1.502300


/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/liam/VSCodeProjects/EDGAR-Summarization/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be re

TrainOutput(global_step=2982, training_loss=1.3935468922038754, metrics={'train_runtime': 25209.6464, 'train_samples_per_second': 0.118, 'train_steps_per_second': 0.118, 'total_flos': 3.411585236415283e+16, 'train_loss': 1.3935468922038754, 'epoch': 2.0})

In [6]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="liamjdavis/phi3-mini-instruct-4k-edgar-summarization", repo_type="model")
api.upload_folder(
    folder_path="phi3-mini-instruct-4k-edgar-summarization",
    repo_id="liamjdavis/phi3-mini-instruct-4k-edgar-summarization",
    repo_type="model"
)

Upload 31 LFS files:   0%|          | 0/31 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

...

events.out.tfevents.1723181845.liamsdell.3836.0:   0%|          | 0.00/5.80k [00:00<?, ?B/s]

events.out.tfevents.1723182230.liamsdell.5907.0:   0%|          | 0.00/68.8k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/liamjdavis/phi3-mini-instruct-4k-edgar-summarization/commit/fb6b8c31e9335044e27b4d9eb9e57e0c7e2ca1c8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='fb6b8c31e9335044e27b4d9eb9e57e0c7e2ca1c8', pr_url=None, pr_revision=None, pr_num=None)